In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

from one.api import ONE

from psychedelic_functions import get_protocol_timings

In [3]:
mode = 'remote'  # sets mode for all queries (one.seach* methods)
one = ONE(mode=mode)

In [4]:
df_infos = pd.read_csv('recordings.csv')
df_infos['date'] = df_infos['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
df_infos['administration_time'] = df_infos['administration_time'].apply(lambda x: np.sum(np.array([int(val) for val in x.split(':')]) * np.array([3600, 60, 1])))

In [5]:
eids = one.search(project='psychedelics', dataset=['spikes', 'Camera'], query_type='local')
print(f'Psychedelics sessions with spike sorting & video: {len(eids)}')

Psychedelics sessions with spike sorting & video: 28


In [9]:
df_timings = get_protocol_timings(one, eids, df_infos=df_infos).set_index('eid')
df_timings.to_csv('protocol_timings.csv')

  4%|████▌                                                                                                                            | 1/28 [00:02<01:12,  2.67s/it]/home/crombie/code/psychedelics/psychedelic_functions.py:115: UserWarning: No stimulus timing found for 86f743cd-ee0d-4ca5-a107-dca9a0cf3f7a, task 01
  warnings.warn(f"No stimulus timing found for {eid}, task {i:02d}")
 14%|██████████████████▍                                                                                                              | 4/28 [00:09<01:01,  2.55s/it]/home/crombie/code/psychedelics/psychedelic_functions.py:100: UserWarning: No intervals table found for 76bae405-b169-44ba-95db-b39342f21cf0, task 01
  warnings.warn(f"No intervals table found for {eid}, task {i:02d}")
 18%|███████████████████████                                                                                                          | 5/28 [00:11<00:49,  2.16s/it]/home/crombie/code/psychedelics/psychedelic_functions.py:100: UserW